In [1]:
using TaylorSeries, TaylorIntegration
using Plots, LaTeXStrings
# plotly()
pyplot(grid = :false, legend = :false, size=(600,600), color=:black, markersize = 0.3 )
const Z = 2.0
const order = 20
const t0 = 0.0
const t_max = 0.5 
const abs_tol = 1.0E-20
const steps = 500_000

500000

In [2]:
function undrivenHelium!(τ, q, dq)
    tT, Q₁, Q₂, P₁, P₂ = q
    
    t1 = Q₁^2
    t2 = Q₂^2
    t = t1 * t2
    dq[1] = t
    R12 = t1 - t2
    aR12 = abs(R12)
    RRR = aR12^3
    c1 = R12/RRR
   
    dq[2] = 0.25*t2*P₁
    dq[4] = 2*Q₁*(-0.125*P₂^2 + Z  - t2 - t2/aR12 + t*c1)
    dq[3] = 0.25*t1*P₂
    dq[5] = 2*Q₂*(-0.125*P₁^2 + Z  - t1 - t1/aR12 - t*c1)

#    return [t, q₁, q₂, p₁, p₂] 
    nothing
end
#undrivenHelium(v) = undrivenHelium(zero(eltype(v)), v)


var1 = set_variables("t q1 q2 p1 p2", order = 1)


function condini(x10::Float64, px10::Float64)
    @assert x10 != 0
    Q1 = sqrt(x10)
    Q2 = 0.0
    P1 = px10*sqrt(x10)    
    P2 = sqrt(8Z)    
    return [0, Q1, Q2, P1, P2]
end

function regHam(t, Q₁, Q₂, P₁, P₂)
    #Cantidades auxiliares
    P₁² = P₁^2
    P₂² = P₂^2
    Q₁² = Q₁^2
    Q₂² = Q₂^2
    nf = abs(Q₁² - Q₂²)
    
    H = 0.125*(P₁²*Q₂² + P₂²*Q₁²) - Z*(Q₁² + Q₂²) + Q₁²*Q₂²*(1.0 + 1.0/nf)
    return H
end


regHam(v) = regHam(v...)

function errHam(N::Int)
    srand(487293456)
    J = [0 0 0 0 0; 0 0 0 1 0; 0 0 0 0 1; 0 -1 0 0 0; 0 0 -1 0 0 ]
    
    dnorm = zeros(N)
    als = typeof(zeros(5))[]
    for j in 1:N
        al = 10*rand(5)
        #al = [BigFloat(x) for x in al1]
        alt = al + var1
        ene = J*∇(regHam(alt))
        ene1 = [ene[k].coeffs[1].coeffs[1] for k in 1:5]
        meq = undrivenHelium(al)
        push!(als, al)
        meq[1] = 0
        dnorm[j] = norm(meq - ene1) 
    end
    return dnorm, als 
end

errHam (generic function with 1 method)

In [3]:
tic()

a1 = condini(rand(2)...)
b1 = a1+var1
x = Array{Taylor1{TaylorN{Float64}}}(5)
for i in eachindex(b1)
    @inbounds x[i] = Taylor1( b1[i], order )
end

ord = x[1].order
#xaux = Array{Taylor1{TaylorN{Float64}}}(5)
#dx = Array{Taylor1{TaylorN{Float64}}}(5)
#for j in eachindex(x)
#    @inbounds xaux[j] = Taylor1( x[j].coeffs[1:ord] )
#end
dx = similar(x)

toc()

elapsed time: 0.279838932 seconds


0.279838932

In [4]:
b1

5-element Array{TaylorSeries.TaylorN{Float64},1}:
                        1.0 t + 𝒪(‖x‖²)
  0.7164577190278739 + 1.0 q1 + 𝒪(‖x‖²)
                       1.0 q2 + 𝒪(‖x‖²)
  0.5309789956120552 + 1.0 p1 + 𝒪(‖x‖²)
                 4.0 + 1.0 p2 + 𝒪(‖x‖²)

In [5]:
x .= Taylor1.( b1, order)

5-element Array{TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}},1}:
                         1.0 t + 𝒪(‖x‖²) + 𝒪(t²¹)
   0.7164577190278739 + 1.0 q1 + 𝒪(‖x‖²) + 𝒪(t²¹)
                        1.0 q2 + 𝒪(‖x‖²) + 𝒪(t²¹)
   0.5309789956120552 + 1.0 p1 + 𝒪(‖x‖²) + 𝒪(t²¹)
                  4.0 + 1.0 p2 + 𝒪(‖x‖²) + 𝒪(t²¹)

In [6]:
x

5-element Array{TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}},1}:
                         1.0 t + 𝒪(‖x‖²) + 𝒪(t²¹)
   0.7164577190278739 + 1.0 q1 + 𝒪(‖x‖²) + 𝒪(t²¹)
                        1.0 q2 + 𝒪(‖x‖²) + 𝒪(t²¹)
   0.5309789956120552 + 1.0 p1 + 𝒪(‖x‖²) + 𝒪(t²¹)
                  4.0 + 1.0 p2 + 𝒪(‖x‖²) + 𝒪(t²¹)

In [7]:
@time undrivenHelium!(0.0, x, dx)

  0.913087 seconds (794.86 k allocations: 37.286 MB, 1.74% gc time)


In [8]:
@code_warntype undrivenHelium!(0.0, x, dx)

Variables:
  #self#::#undrivenHelium!
  τ::Float64
  q::Array{TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}},1}
  dq::Array{TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}},1}
  tT::TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}}
  Q₁::TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}}
  Q₂::TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}}
  P₁::TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}}
  P₂::TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}}
  #temp#::Int64
  t1::TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}}
  t2::TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}}
  t::TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}}
  R12::TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}}
  aR12::TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}}
  RRR::TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}}
  c1::TaylorSeries.Taylor1{TaylorSeries.TaylorN{Float64}}

Body:
  begin 
      #temp#::Int64 = $(QuoteNode(1))
      SSAValue(31) = (Base.arrayref)(q::Array{T

In [9]:
sizeof(x[2])

16

In [10]:
@time undrivenHelium!(0.0, x, dx)

  0.035569 seconds (285.67 k allocations: 15.986 MB, 33.57% gc time)


In [ ]:
x